In [ ]:
from google.colab import drive
drive.mount("/content/drive")


!pip install unrar
!unrar x /content/drive/MyDrive/data.rar




import os
import numpy as np
import librosa
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
# Constants
DATASET_PATH = "/content/data"
NUM_CLASSES = 2  # Assuming there are two classes, you can adjust this based on your folder structure
SAMPLE_RATE = 16000
DURATION = 5
N_MELS = 128
max_time_steps = 109
# Dynamically generate labels based on folder names
labels = {}
class_index = 0

for root, dirs, files in os.walk(DATASET_PATH):
    for folder in dirs:
        folder_path = os.path.join(root, folder)

        # Assuming each folder is a class, you may need to adjust this based on your specific dataset structure
        labels[folder] = class_index
        class_index += 1

# Load audio data and labels
X = []
y = []

for root, dirs, files in os.walk(DATASET_PATH):
    for file_name in files:
        file_path = os.path.join(root, file_name)

        audio, _ = librosa.load(file_path, sr=SAMPLE_RATE, duration=DURATION)

        mel_spectrogram = librosa.feature.melspectrogram(y=audio, sr=SAMPLE_RATE, n_mels=N_MELS)
        mel_spectrogram = librosa.power_to_db(mel_spectrogram, ref=np.max)

        if mel_spectrogram.shape[1] < max_time_steps:
            mel_spectrogram = np.pad(mel_spectrogram, ((0, 0), (0, max_time_steps - mel_spectrogram.shape[1])), mode='constant')
        else:
            mel_spectrogram = mel_spectrogram[:, :max_time_steps]

        X.append(mel_spectrogram)
        # Extract label from the folder name
        label = labels[root.split("/")[-1]]  # Adjust this based on your specific folder structure
        y.append(label)

# Convert to numpy arrays
X = np.array(X)
y = np.array(y)

# One-hot encode labels
y_encoded = to_categorical(y, NUM_CLASSES)

# Split the data into training and validation sets
split_index = int(0.8 * len(X))
X_train, X_val = X[:split_index], X[split_index:]
y_train, y_val = y_encoded[:split_index], y_encoded[split_index:]

In [ ]:

# Define the model
model_input = Input(shape=(N_MELS, max_time_steps, 1))
x = Conv2D(32, kernel_size=(3, 3), activation='relu')(model_input)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Conv2D(64, kernel_size=(3, 3), activation='relu')(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
model_output = Dense(NUM_CLASSES, activation='softmax')(x)

# Compile the model
model = Model(inputs=model_input, outputs=model_output)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


model.fit(X_train, y_train, batch_size=32, epochs=10, validation_data=(X_val, y_val))

# Save the model
model.save("Deepfake_audio.h5")

Epoch 1/10
500/500 [==============================] - 14s 14ms/step - loss: 2.2868 - accuracy: 0.6212 - val_loss: 0.9814 - val_accuracy: 0.0000e+00
Epoch 2/10
500/500 [==============================] - 7s 14ms/step - loss: 0.6621 - accuracy: 0.6248 - val_loss: 1.0061 - val_accuracy: 0.0000e+00
Epoch 3/10
500/500 [==============================] - 7s 14ms/step - loss: 0.6624 - accuracy: 0.6250 - val_loss: 0.9714 - val_accuracy: 0.0000e+00
Epoch 4/10
500/500 [==============================] - 7s 13ms/step - loss: 0.6624 - accuracy: 0.6249 - val_loss: 0.9723 - val_accuracy: 0.0000e+00
Epoch 5/10
500/500 [==============================] - 7s 14ms/step - loss: 0.6621 - accuracy: 0.6250 - val_loss: 0.9761 - val_accuracy: 0.0000e+00
Epoch 6/10
500/500 [==============================] - 7s 13ms/step - loss: 0.6622 - accuracy: 0.6250 - val_loss: 0.9772 - val_accuracy: 0.0000e+00
Epoch 7/10
500/500 [==============================] - 7s 14ms/step - loss: 0.6620 - accuracy: 0.6250 - val_loss: 0.97

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import os
import numpy as np
import librosa
import tensorflow as tf
from tensorflow.keras.models import load_model

# Define paths and parameters
TEST_DATASET_PATH = "/content/Test"
MODEL_PATH = "Deepfake_audio.h5"  # Replace with the actual path to your saved model
SAMPLE_RATE = 16000
DURATION = 5
N_MELS = 128
MAX_TIME_STEPS = 109

# Load the saved model
model = load_model(MODEL_PATH)

# Load and preprocess test data using librosa
X_test = []

test_files = os.listdir(TEST_DATASET_PATH)
for file_name in test_files:
    file_path = os.path.join(TEST_DATASET_PATH, file_name)

    try:
        # Check if the file is an audio file (skip non-audio files)
        if not file_name.endswith(('.wav', '.mp3', '.flac')):
            continue

        # Load audio file using librosa
        audio, _ = librosa.load(file_path, sr=SAMPLE_RATE, duration=DURATION)

        # Extract Mel spectrogram using librosa
        mel_spectrogram = librosa.feature.melspectrogram(y=audio, sr=SAMPLE_RATE, n_mels=N_MELS)
        mel_spectrogram = librosa.power_to_db(mel_spectrogram, ref=np.max)

        # Ensure all spectrograms have the same width (time steps)
        if mel_spectrogram.shape[1] < MAX_TIME_STEPS:
            mel_spectrogram = np.pad(mel_spectrogram, ((0, 0), (0, MAX_TIME_STEPS - mel_spectrogram.shape[1])), mode='constant')
        else:
            mel_spectrogram = mel_spectrogram[:, :MAX_TIME_STEPS]

        X_test.append(mel_spectrogram)
    except Exception as e:
        print(f"Error processing file {file_name}: {e}")

# Convert list to numpy array
X_test = np.array(X_test)

# Predict using the loaded model
y_pred = model.predict(X_test)

# Convert probabilities to predicted classes
y_pred_classes = np.argmax(y_pred, axis=1)

# Print the predicted classes
print("Predicted classes:", y_pred_classes)


1/1 [==============================] - 0s 87ms/step
Predicted classes: [0 1 0 0 0 1 1 1 1 0]
